In [1]:
library(readxl)
data <- read_excel("longley.xlsx")
head(data)

GNP.deflator,GNP,Unemployed,Armed.Forces,Population,Year,Employed
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
83.0,234.289,235.6,159.0,107.608,1947,60.323
88.5,259.426,232.5,145.6,108.632,1948,61.122
88.2,258.054,368.2,161.6,109.773,1949,60.171
89.5,284.599,335.1,165.0,110.929,1950,61.187
96.2,328.975,209.9,309.9,112.075,1951,63.221
98.1,346.999,193.2,359.4,113.270,1952,63.639


判断共线性的严重程度

In [2]:
# 提取前六列
X <- data[, 1:6]

# 中心化和标准化
X_scaled <- scale(X)

# 计算 X'X
XTX <- t(X_scaled) %*% X_scaled

# 计算特征值
eig_values <- eigen(XTX)$values

# 按从大到小排序
sorted_eig_values <- sort(eig_values, decreasing = TRUE)

#计算条件数
k=sorted_eig_values[1]/sorted_eig_values[6]

eig_values

[1] 69.050656437 17.630107489  3.051380586  0.223923880  0.038280986
[6]  0.005650622

In [3]:
k

[1] 12220.01

用PCA来解决复共线性，并选择恰当个数的主成分

In [4]:
# 提取前六列
X <- data[, 1:6]

# 对X进行PCA变换
pca_result <- prcomp(X, center = TRUE, scale. = TRUE)

# 查看PCA的结果，包括主成分得分和主成分载荷
summary(pca_result)



Importance of components:
                          PC1    PC2    PC3     PC4     PC5     PC6
Standard deviation     2.1455 1.0841 0.4510 0.12218 0.05052 0.01941
Proportion of Variance 0.7672 0.1959 0.0339 0.00249 0.00043 0.00006
Cumulative Proportion  0.7672 0.9631 0.9970 0.99951 0.99994 1.00000

选两个主成分，做回归，并恢复到原来的变量

In [5]:
Y <- as.numeric(data$Employed)

# 主成分得分（PCA变换后的数据）
pca_scores <- pca_result$x[, 1:2]

# 对Y进行回归分析，使用前两个主成分
regression_model <- lm(Y ~ pca_scores)

# 查看回归模型结果
summary(regression_model)


Call:
lm(formula = Y ~ pca_scores)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.83922 -0.52416  0.04553  0.74571  1.62193 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)    65.3170     0.2515 259.703  < 2e-16 ***
pca_scoresPC1   1.5651     0.1211  12.928 8.51e-09 ***
pca_scoresPC2  -0.3918     0.2396  -1.635    0.126    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 1.006 on 13 degrees of freedom
Multiple R-squared:  0.9289,	Adjusted R-squared:  0.9179 
F-statistic:  84.9 on 2 and 13 DF,  p-value: 3.45e-08


In [6]:
# 提取回归系数
coefficients <- regression_model$coefficients

# 恢复到原始变量空间
# 通过PCA载荷矩阵（主成分载荷矩阵）来恢复系数
pca_loadings <- pca_result$rotation[, 1:2]

# 计算标准化后的系数 beta
beta_standardized <- pca_loadings %*% coefficients[2:3]

# 提取原始数据的均值和标准差
X_means <- colMeans(X)
X_sds <- apply(X, 2, sd)

# 将系数恢复到原始变量空间
beta_original <- beta_standardized / X_sds

# 恢复截距项
original_intercept <- coefficients[1] - sum((X_means / X_sds) * beta_standardized)

# 输出恢复后的系数
list(
  original_intercept = original_intercept,
  original_coefficients = beta_original
)


GNP.deflator,0.069080693
GNP,0.007476802
Unemployed,0.002884626
Armed.Forces,0.009026034
Population,0.101446710
Year,0.152895109


岭回归

In [7]:
# 先定义 ridge_regression 函数
ridge_regression <- function(X, Y, k) {
  # 确保 Y 是数值型向量
  Y <- as.numeric(Y)

  # 对 X 进行中心化和标准化
  X_scaled <- scale(X)

  # 执行岭回归，alpha=0 表示岭回归（L2 正则化）
  ridge_model <- glmnet(X_scaled, Y, alpha = 0, lambda = k, standardize = FALSE)

  # 提取标准化后的系数（包含截距）
  beta_standardized <- coef(ridge_model)

  # 将系数转换为数值向量
  beta_standardized <- as.vector(beta_standardized)

  # 计算预测值
  Y_pred <- cbind(1, X_scaled) %*% beta_standardized  # 计算预测值

  # 计算残差平方和 (RSS)
  RSS <- sum((Y - Y_pred) ^ 2)

  # 恢复到原始变量空间的系数
  X_means <- attr(X_scaled, "scaled:center")
  X_sds <- attr(X_scaled, "scaled:scale")

  beta_original <- beta_standardized[-1] / X_sds  # 去掉截距项并进行恢复
  intercept_original <- beta_standardized[1] - sum(X_means * beta_original)

  # 返回结果
  return(list(
    beta_standardized = beta_standardized,
    beta_original = c(intercept_original, beta_original),
    RSS = RSS  # 返回 RSS 值
  ))
}

H-K公式选取k

In [8]:
X <- data[, 1:6]

# 确保Y是数值型向量
Y <- as.numeric(data$Employed)

# 标准化 X
X_scaled <- scale(X)

# 对 X'X 进行特征值分解 (spectral decomposition)
XtX <- t(X_scaled) %*% X_scaled
eigen_decomp <- eigen(XtX)

# 提取特征向量矩阵 Phi 和特征值矩阵 Lambda
Phi <- eigen_decomp$vectors
Lambda <- diag(eigen_decomp$values)

# 计算新变量矩阵 Z = X * Phi
Z <- X_scaled %*% Phi

# 进行典则形式的回归 y = alpha_0 + Z * alpha + e
canonical_model <- lm(Y ~ ., data = as.data.frame(Z))

# 提取回归系数 alpha（去掉截距项）
alpha <- coef(canonical_model)[-1]

# 计算残差的方差 sigma^2
residuals <- residuals(canonical_model)
sigma_squared <- var(residuals)

# 选取 alpha 的最大值
max_alpha2 <- max(alpha**2)

# 根据 H-K 公式计算岭回归系数 k
k <- sigma_squared / max_alpha2
k

[1] 0.001149996

In [9]:
sigma_squared

[1] 0.0557616

In [10]:
summary(canonical_model)


Call:
lm(formula = Y ~ ., data = as.data.frame(Z))

Residuals:
     Min       1Q   Median       3Q      Max 
-0.41011 -0.15767 -0.02816  0.10155  0.45539 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 65.31700    0.07621 857.026  < 2e-16 ***
V1          -1.56511    0.03669 -42.662 1.07e-11 ***
V2           0.39183    0.07260   5.397 0.000435 ***
V3           1.86039    0.17452  10.660 2.10e-06 ***
V4           0.35730    0.64423   0.555 0.592672    
V5           6.16983    1.55812   3.960 0.003305 ** 
V6           6.96337    4.05550   1.717 0.120105    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.3049 on 9 degrees of freedom
Multiple R-squared:  0.9955,	Adjusted R-squared:  0.9925 
F-statistic: 330.3 on 6 and 9 DF,  p-value: 4.984e-10


In [18]:
library(glmnet)
# 定义岭迹图绘制函数
ridge_trace <- function(X, Y, k_values) {
  # 创建一个矩阵以存储标准化系数
  beta_matrix <- matrix(0, nrow = length(k_values), ncol = ncol(X) + 1)

  # 计算每个 k 值下的标准化系数
  for (i in seq_along(k_values)) {
    k <- k_values[i]
    ridge_result <- ridge_regression(X, Y, k)
    
    # 将标准化系数存入矩阵
    beta_matrix[i, ] <- ridge_result$beta_standardized
  }
  
  # 将 k_values 转换为矩阵（确保是列矩阵）
  k_values_matrix <- matrix(k_values, ncol = 1)

  # 打开图形设备
  png("ridge_trace_plot2031.jpg", width = 800, height = 600)

  # 绘制岭迹图，不包含截距项的系数
  matplot(k_values_matrix, beta_matrix[, -1], type = "l", lty = 1, col = 1:ncol(X), 
          xlab = "岭回归系数 k", ylab = "标准化系数", 
          main = "岭迹图", ylim = range(beta_matrix[, -1]), lwd = 2)

  # 添加图例，确保与X的列名匹配
  legend("topright", legend = colnames(X), col = 1:ncol(X), lty = 1)
  
  # 添加水平线
  abline(h = 0, col = "gray", lwd = 2)
  
  # 关闭图形设备
  dev.off()
}
# 使用示例
k_values <- seq(1, 4, length.out =3000 ) 
ridge_trace(X = data[, 1:6], Y = data$Employed, k_values = k_values)

png 
  2

In [51]:
## k_values <- seq(0.01, 5, length.out = 500)  # 生成 100 个 k 值
ridge_trace(X = data[, 1:6], Y = data$Employed, k_values = k_values)

png 
  2

png 
  2

In [35]:
k_values <- seq(0.01, 4, length.out =400 )  # 生成 100 个 k 值
RSS_matrix <- matrix(0, nrow = length(k_values), ncol = 2)
for (i in seq_along(k_values)) {
    k <- k_values[i]
    ridge_result <- ridge_regression(X, Y, k)
    
    # 将标准化系数存入矩阵
    RSS_matrix[i, 1 ] <- k
    RSS_matrix[i, 2 ] <- ridge_result$RSS
  }
RSS_matrix[300,]

[1]  3.00000 15.13089

In [29]:
RSS_matrix[1:10,]

0.01,1.522360
0.02,1.839383
0.03,2.020262
0.04,2.142936
0.05,2.235747
0.06,2.312358
0.07,2.379312
0.08,2.440392
0.09,2.497778
0.10,2.552945


In [39]:
ridge_regression(X = data[, 1:6], Y = data$Employed, k = 3)

$beta_standardized
[1] 65.31700000  0.66160839  0.70302806  0.04679752  0.24896327  0.64372105
[7]  0.65986924

$beta_original
               GNP.deflator           GNP    Unemployed  Armed.Forces 
-2.265122e+02  6.130798e-02  7.073077e-03  5.007952e-04  3.577472e-03 
   Population          Year 
 9.254049e-02  1.386003e-01 

$RSS
[1] 15.13089

In [40]:
ridge_regression(X = data[, 1:6], Y = data$Employed, k = 0)


$beta_standardized
[1] 65.31700000 -0.08574584 -2.14126208 -1.69910570 -0.68156796 -0.83364414
[7]  7.87582912

$beta_original
               GNP.deflator           GNP    Unemployed  Armed.Forces 
-3.136333e+03 -7.945644e-03 -2.154297e-02 -1.818267e-02 -9.793774e-03 
   Population          Year 
-1.198436e-01  1.654255e+00 

$RSS
[1] 0.8541842

In [41]:
ridge_regression(X = data[, 1:6], Y = data$Employed, k = 0.00115)

$beta_standardized
[1] 65.3170000 -0.1477884 -0.2707759 -1.4272978 -0.6140348 -1.1183113  6.1465903

$beta_original
               GNP.deflator           GNP    Unemployed  Armed.Forces 
-2.429521e+03 -1.369482e-02 -2.724242e-03 -1.527397e-02 -8.823358e-03 
   Population          Year 
-1.607670e-01  1.291042e+00 

$RSS
[1] 0.9679234

In [42]:
# 确保 Y 是数值型向量
Y <- as.numeric(data$Employed)

# 创建模型，直接对 Y 和 X 做回归
model <- lm(Y ~ ., data = data.frame(X))

# 查看模型摘要
summary(model)


Call:
lm(formula = Y ~ ., data = data.frame(X))

Residuals:
     Min       1Q   Median       3Q      Max 
-0.41011 -0.15767 -0.02816  0.10155  0.45539 

Coefficients:
               Estimate Std. Error t value Pr(>|t|)    
(Intercept)  -3.482e+03  8.904e+02  -3.911 0.003560 ** 
GNP.deflator  1.506e-02  8.492e-02   0.177 0.863141    
GNP          -3.582e-02  3.349e-02  -1.070 0.312681    
Unemployed   -2.020e-02  4.884e-03  -4.136 0.002535 ** 
Armed.Forces -1.033e-02  2.143e-03  -4.822 0.000944 ***
Population   -5.110e-02  2.261e-01  -0.226 0.826212    
Year          1.829e+00  4.555e-01   4.016 0.003037 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.3049 on 9 degrees of freedom
Multiple R-squared:  0.9955,	Adjusted R-squared:  0.9925 
F-statistic: 330.3 on 6 and 9 DF,  p-value: 4.984e-10
